# Shortest Path Problem

## Graph representation

In [ ]:
import math
import numpy as np

n_nodes = 10

# List of tuples (start, destination, cost) which represent directed edges
edges_list = []
edges_list.append((0, 1, 2))
edges_list.append((0, 2, 10))
edges_list.append((0, 3, 7))
edges_list.append((1, 5, 4))
edges_list.append((2, 3, -4))
edges_list.append((2, 4, 3))
edges_list.append((4, 0, -5))
edges_list.append((5, 3, 12))
edges_list.append((7, 8, 3))
edges_list.append((8, 9, 4))

# Adjacency matrix representation of a graph of n_nodes nodes
# Initialize all values to infinite
graph_matrix = np.full((n_nodes, n_nodes), np.inf) 
# Values in the diagonal are 0: going from any node to itself has no cost
np.fill_diagonal(graph_matrix, 0)

for edge in edges_list:
    start, destination, cost = edge
    graph_matrix[start, destination] = cost

## All Pairs Shorterst Path

### Floyd Warshall Algorithm

In [ ]:
"""
    Takes an adjacency matrix representing a graph and returns a new matrix
    with matrix[i, j] containg the shortest path between node i and node j.
"""
def floyd_warshall(graph):
    g = grap[:][:]
    n_nodes = len(g)
    floyd_warshall_helper(g, n_nodes - 1)
    
    return g

""" 
    Takes a matrix representing a graph and modifies that matrix
    so that at the end of the function graph[i, j] contains the shortest path between
    node i and node j using only nodes in the set {0,1, .., k} as intermediary nodes, 
    which we will call sp(i, j, k).
      We have sp(i, j, k) = min(sp(i, j, k - 1), sp(i, k, k - 1) + sp(k, j, k - 1)).
    This is saying that the shortest path from i to j using nodes {0, 1, .., k} is 
    the minimum between going from i to j using only nodes {0,1, .., k - 1}, and the
    sum of first going from i to k and then from j to k (both steps still with nodes {0,1, .., k - 1}).
"""
def floyd_warshall_helper(graph, k):
    if k == -1:
        # No intermediary node, leave the original adjacency matrix unchanged
        return
    
    # Run recursively
    floyd_warshall_helper(graph, k - 1)
    # Now graph[i][j] contains the shortest path using node {0, 1, .., k - 1} as intermediary
    
    n_nodes = len(g)
    # Update distance to get from i to j, for every i and j
    for i in n_nodes:
        for j in n_nodes:
            graph[i][j] = min(graph[i][j], graph[i][k] + graph[k][j])